In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

In [3]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [4]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30000000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [5]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'irproject-414719'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"

Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [6]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd

**Count the frequency of each word in docunment**

In [7]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)
stemmer = PorterStemmer()

"""
Count the frequency of each word in docunment
Parameters:
  text: (str) Text of one document
  id: (int) Document id
  withTFnormal (bool) if normalization by leanght of the document
Returns:
  List of tuples: A list of (token, (doc_id, tf))
"""

stemDict = {}
l = ["barcelona", "ring"]

def word_count(text, id):
  containl = False
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  sizeOfDoc = 0
  word_counts = Counter()

  for token in tokens:
      if (token not in all_stopwords):
          if(token in stemDict):
              word_counts[token] += 1
          else:
              stemDict[token] = stemmer.stem(token)
              word_counts[stemDict[token]] += 1

          sizeOfDoc += 1
          if(containl == False):
              if(token in l):
                  containl = True

  if len(stemDict) >= 20000:
      stemDict.clear()

  if(containl == False):
      return []

  return [(token, (id, count, sizeOfDoc)) for token, count in word_counts.items()]

In [8]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))    # Count the frequency of each word in docunment without normalization
word_counts_withoutSize = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1])))

**sorted posting list by wiki_id**

In [9]:
"""
Returns a sorted posting list by wiki_id.
Parameters:
  unsorted_pl: list of tuples - A list of (wiki_id, tf) tuples
Returns:
    list of tuples A sorted posting list.
"""
def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

In [10]:
postings = word_counts_withoutSize.groupByKey().mapValues(reduce_word_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.filter(lambda x: len(x[1])>10)         # Filter out rare words

**Takes a posting list RDD and calculate the df for each token**

In [11]:
"""
Takes a posting list RDD and calculate the df for each token.
Parameters:
  postings: (RDD) An RDD where each element is a (token, posting_list).
Returns:
  An RDD where each element is a (token, df) pair.
"""
def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

In [12]:
w2df_dict = (calculate_df(postings_filtered)).collectAsMap()    # Create dict of df

**A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk**

In [13]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

"""
A function that partitions the posting lists into buckets, writes out all posting lists in a bucket to disk.
Parameters:
  postings: (RDD) An RDD where each item is a (w, posting_list) pair.
Returns:
  An RDD where each item is a posting locations dictionary for a bucket.
"""
def partition_postings_and_write(postings):

  bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         # Assign each token to a bucket
  grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        # Group the postings by bucket
  posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x))   # Write out each bucket and collect posting locations

  return posting_locations

In [14]:
# Merge the posting locations into a single dict.
posting_locs_list = partition_postings_and_write(postings).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

**Create tf-idf dictionary**

In [15]:
# Create tf-idf dictionary

N = 1000;

word_TF = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1]/x[1][2])))
postings_TF = word_TF.groupByKey().mapValues(reduce_word_counts)
postings_TF_filtered = postings_TF.filter(lambda x: len(x[1])>10)

postings_TF_IDF_filtered = postings_TF_filtered.map(lambda x: (x[0], updated_value_function(x[0],x[1])))

def updated_value_function(term, post):
    listTermTfIdF = []

    if (term in w2df_dict):
        for doc, tf in post:
            listTermTfIdF.append((doc, tf * math.log10(N / w2df_dict[term])))

    return listTermTfIdF

postings_TF_IDF_filtered_dict = postings_TF_IDF_filtered.collectAsMap()

{'polit': [(12, 0.000889787359885655), (600, 0.0003557612673808793), (689, 0.0004056612613395541), (771, 0.0001631929546057889), (783, 0.00023586157306892646), (1203, 0.00039119475929980205), (1273, 0.0004463397049047218), (1556, 0.0007885461353984526), (1676, 0.0014795466594746706), (1679, 0.0013318045720702313), (1719, 0.0011933517633226664), (1806, 0.00026107195667113953), (1990, 0.0005447757755867652), (2148, 0.00036959169942618863)], 'movement': [(12, 0.00885587906177707), (600, 0.0002950685093266252), (657, 0.00033360859234129835), (674, 0.0012333705899206235), (679, 0.0026419278386615873), (680, 0.0008614016818493243), (689, 0.000336455580328456), (737, 0.0003725262270384116), (738, 0.00016680429617064917), (771, 0.0001353522889173454), (808, 0.00020739169167804746), (844, 0.00039937714340286854), (1029, 0.0007887160821241568), (1057, 0.0018525325326518604), (1098, 0.0005018283296674425), (1141, 0.000420071578754497), (1203, 0.0006489141177890035), (1315, 0.0007314947678882226),

**Create nf dictionary**

In [16]:
Doc_TF = word_counts.map(lambda x: (x[1][0], (x[0], x[1][1]/x[1][2])))
Doc_TF = Doc_TF.groupByKey().mapValues(reduce_word_counts)

Doc_TF_IDF_filtered = Doc_TF.map(lambda x: (x[0], updated_value_function2(x[1])))
def updated_value_function2(post):
  listTermTfIdF = []

  for term, tf in post:
      if (term in w2df_dict):
          listTermTfIdF.append((term, tf * math.log10(N / w2df_dict[term])))

  nf = 0
  for term, tfIdf in listTermTfIdF:
      nf = nf + tfIdf ** 2
  nf = 1/math.sqrt(nf)

  return nf

Doc_TF_IDF_filtered_dict = Doc_TF_IDF_filtered.collectAsMap()

**write the global stats out**

In [17]:
inverted = InvertedIndex()                            # Create inverted index instance
inverted.posting_locs = super_posting_locs            # Adding the posting locations dictionary to the inverted index
inverted.df = w2df_dict                               # Add the df dictionary to the inverted index
inverted.tfidf = postings_TF_IDF_filtered_dict        # Add the tfidf dictionary to the inverted index
inverted.nf = Doc_TF_IDF_filtered_dict
inverted.write_index('.', 'index')                    # write the global stats out


In [18]:
inverted_index = InvertedIndex.read_index('.', 'index')

**--------------------------------------------------- Title -----------------------------------------------------------**

In [129]:
parquetFile = spark.read.parquet(path)
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("title", "id").rdd

In [123]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)
stemmer = PorterStemmer()


stemDict = {}
l = ["telecommunications", "anarchism"]

def word_title_count(title, id):
  containl = False
  tokens = [token.group() for token in RE_WORD.finditer(title.lower())]
  sizeOfTitle = 0
  word_counts = Counter()

  for token in tokens:
      if (token not in all_stopwords):
          if(token in stemDict):
              word_counts[token] += 1
          else:
              stemDict[token] = stemmer.stem(token)
              word_counts[stemDict[token]] += 1

          sizeOfTitle += 1
          if(containl == False):
            if(token in l):
              containl = True

  if len(stemDict) >= 20000:
      stemDict.clear()

  if(containl == False and id != 0):
      return []

  return [(token, (id, count/sizeOfTitle)) for token, count in word_counts.items()]

In [124]:
def reduce_word_title_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

In [125]:
word_counts = doc_text_pairs.flatMap(lambda x: word_title_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_title_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.collectAsMap()
word_counts.take(1)

[('anarch', (12, 1.0))]

In [126]:
inverted = InvertedIndex()
inverted.tfTitle = postings_filtered
inverted.write_index('.', 'index')

In [127]:
inverted_index = InvertedIndex.read_index('.', 'index')

In [128]:
simDoc = Counter()

query = stemmer.stem("Anarchism")
query = dict(word_title_count(query, 0))
for term,  value in query.items():
    docTfItf = inverted_index.tfTitle[term]
    for doc, weight in docTfItf:
        simDoc[doc] += value[1] * weight


top_10_similar_docs = simDoc.most_common(10)
top_10_similar_docs

[(12, 1.0)]